In [1]:
# MLP with automatic validation set
from keras.models import Sequential
from keras.layers import Dense
import numpy
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from functools import reduce
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import pandas as pd
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import models
from keras import layers
from keras import optimizers
from keras import initializers
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 6
numpy.random.seed(seed)

In [3]:
# load pima indians dataset
#dataset = numpy.loadtxt("D:/Neural_nets_course_Dataset/pima_indians_diabetes_data.csv", delimiter=",")
dataset = pd.read_csv('D:/Neural_nets_course_Dataset/ucl_credit_dataset.csv', delimiter=',', names = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','y'])

In [4]:
dataset.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,y
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [5]:
dataset.isnull().sum()

a    0
b    0
c    0
d    0
e    0
f    0
g    0
h    0
i    0
j    0
k    0
l    0
m    0
n    0
o    0
y    0
dtype: int64

In [6]:
dataset['e'].mode()[0]

'g'

In [7]:
dataset['a'].value_counts()

b    468
a    210
?     12
Name: a, dtype: int64

In [8]:
dataset.isnull().sum()

a    0
b    0
c    0
d    0
e    0
f    0
g    0
h    0
i    0
j    0
k    0
l    0
m    0
n    0
o    0
y    0
dtype: int64

In [9]:
# dataset['a'] = dataset['a'].replace('?', numpy.nan)
# dataset['b'] = dataset['b'].replace('?', numpy.nan)
# dataset['e'] = dataset['e'].replace('?', numpy.nan)
# dataset['f'] = dataset['f'].replace('?', numpy.nan)
# dataset['g'] = dataset['g'].replace('?', numpy.nan)
# dataset['b'] = dataset['b'].replace('?', numpy.nan)
dataset = dataset.replace('?', numpy.nan)

In [10]:
len(dataset)

690

In [11]:
dataset = dataset.dropna()

In [12]:
len(dataset)

653

In [13]:
#dataset.loc[dataset['a'] =='?', 'a'] = dataset['a'].mode()[0]
dataset.loc[dataset['a'] == 'a', 'a'] = 1
dataset.loc[dataset['a'] == 'b', 'a'] = 2

In [14]:
#dataset.loc[dataset['i'] =='?', 'i'] = dataset['i'].mode()[0]
dataset.loc[dataset['i'] == 't', 'i'] = 1
dataset.loc[dataset['i'] == 'f', 'i'] = 0

In [15]:
dataset.loc[dataset['j'] == 't', 'j'] = 1
dataset.loc[dataset['j'] == 'f', 'j'] = 0

dataset.loc[dataset['l'] == 't', 'l'] = 1
dataset.loc[dataset['l'] == 'f', 'l'] = 0

dataset.loc[dataset['m'] == 'g', 'm'] = 1
dataset.loc[dataset['m'] == 's', 'm'] = 2
dataset.loc[dataset['m'] == 'p', 'm'] = 3

In [16]:
#dataset.loc[dataset['d'] =='?', 'd'] = dataset['d'].mode()[0]
dataset.loc[dataset['d'] == 'u', 'd'] = 1
dataset.loc[dataset['d'] == 'y', 'd'] = 2
dataset.loc[dataset['d'] == 'l', 'd'] = 3

In [17]:
#dataset.loc[dataset['e'] =='?', 'e'] = dataset['e'].mode()[0]
dataset.loc[dataset['e'] == 'g', 'e'] = 1
dataset.loc[dataset['e'] == 'p', 'e'] = 2
dataset.loc[dataset['e'] == 'gg', 'e'] = 3

In [18]:
#dataset.loc[dataset['f'] =='?', 'f'] = dataset['f'].mode()[0]
dataset.loc[dataset['f'] == 'c', 'f'] = 1
dataset.loc[dataset['f'] == 'q', 'f'] = 2
dataset.loc[dataset['f'] == 'w', 'f'] = 3
dataset.loc[dataset['f'] == 'i', 'f'] = 4
dataset.loc[dataset['f'] == 'aa', 'f'] = 5
dataset.loc[dataset['f'] == 'ff', 'f'] = 6
dataset.loc[dataset['f'] == 'k', 'f'] = 7
dataset.loc[dataset['f'] == 'cc', 'f'] = 8
dataset.loc[dataset['f'] == 'x', 'f'] = 9
dataset.loc[dataset['f'] == 'm', 'f'] = 10
dataset.loc[dataset['f'] == 'd', 'f'] = 11
dataset.loc[dataset['f'] == 'e', 'f'] = 12
dataset.loc[dataset['f'] == 'j', 'f'] = 13
dataset.loc[dataset['f'] == 'r', 'f'] = 14

In [19]:
#dataset.loc[dataset['g'] =='?', 'g'] = dataset['g'].mode()[0]
dataset.loc[dataset['g'] == 'v', 'g'] = 1
dataset.loc[dataset['g'] == 'h', 'g'] = 2
dataset.loc[dataset['g'] == 'bb', 'g'] = 3
dataset.loc[dataset['g'] == 'ff', 'g'] = 4
dataset.loc[dataset['g'] == 'j', 'g'] = 5
dataset.loc[dataset['g'] == 'z', 'g'] = 6
dataset.loc[dataset['g'] == 'dd', 'g'] = 7
dataset.loc[dataset['g'] == 'n', 'g'] = 8
dataset.loc[dataset['g'] == 'o', 'g'] = 9

In [20]:
dataset['a'].unique()

array([2, 1], dtype=int64)

In [21]:
#dataset['b'] = dataset['b'].replace('?', numpy.nan)
# dataset['b'] = dataset['b'].astype(float)
# dataset.loc[:, 'b'] = dataset.fillna(dataset.mean()['b'])

In [22]:
# dataset['c'] = dataset['c'].replace('?', numpy.nan)
# dataset['c'] = dataset['c'].astype(float)
# dataset.loc[:, 'c'] = dataset.fillna(dataset.mean()['c'])

# dataset['h'] = dataset['h'].replace('?', numpy.nan)
# dataset['h'] = dataset['h'].astype(float)
# dataset.loc[:, 'h'] = dataset.fillna(dataset.mean()['h'])

In [23]:
dataset.loc[dataset['y'] =='+', 'y'] = 1
dataset.loc[dataset['y'] =='-', 'y'] = 0

In [24]:
# dataset.head()
# dataset.__delitem__('n')

In [25]:
dataset.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,y
0,2,30.83,0.000,1,1,3,1,1.25,1,1,1,0,1,00202,0,1
1,1,58.67,4.460,1,1,2,2,3.04,1,1,6,0,1,00043,560,1
2,1,24.50,0.500,1,1,2,2,1.50,1,0,0,0,1,00280,824,1
3,2,27.83,1.540,1,1,3,1,3.75,1,1,5,1,1,00100,3,1
4,2,20.17,5.625,1,1,3,1,1.71,1,0,0,0,2,00120,0,1


In [26]:
# split into input (X) and output (Y) variables
X = dataset.iloc[:,0:15].values
Y = dataset.iloc[:,15].values

In [27]:
X

array([[2, '30.83', 0.0, ..., 1, '00202', 0],
       [1, '58.67', 4.46, ..., 1, '00043', 560],
       [1, '24.50', 0.5, ..., 1, '00280', 824],
       ...,
       [1, '25.25', 13.5, ..., 1, '00200', 1],
       [2, '17.92', 0.205, ..., 1, '00280', 750],
       [2, '35.00', 3.375, ..., 1, '00000', 0]], dtype=object)

In [ ]:
# define 5-fold cross validation test harness
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=seed)
cvscores = []
trainScores = []
f1Score = []
num_k_folds = 5
fold_counter = 0
val_conmats = []
val_precisions = []
val_recalls = []
val_f_scores = []
input_dim = 15

In [35]:
# create model
model = Sequential()
model.add(Dense(15, input_dim=input_dim, activation='relu', kernel_initializer=initializers.he_normal(seed=None)))
model.add(Dense(15, activation='relu', kernel_initializer=initializers.he_normal(seed=None)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 16)                256       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [37]:
for train, test in kfold.split(X, Y):
    # Fit the model
    model.fit(X[train], Y[train], epochs=400, batch_size=16, verbose=0)
    
    y_train_pred = model.predict(X[train])
    y_train_pred = numpy.asarray([1 if val >= 0.5 else 0 for val in y_train_pred], dtype=numpy.int64)
    
    y_validation_pred = model.predict(X[test])
    y_validation_pred = numpy.asarray([1 if val >= 0.5 else 0 for val in y_validation_pred], dtype=numpy.int64)
    #y_validation_pred = np.argmax(y_validation_pred, axis=1)
    
    [precision, recall, f_score, _] = precision_recall_fscore_support(Y[test], y_validation_pred)
    #print("Validation k-fold #%d - precision: %f, recallL: %f, f-score: %f" % (fold_counter, precision, recall, f_score))
    
    conmat = confusion_matrix(Y[test], y_validation_pred)
    
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_f_scores.append(f_score)
    val_conmats.append(conmat)
    fold_counter = fold_counter + 1
    
    trainScores.append(accuracy_score(Y[train], y_train_pred))
    cvscores.append(accuracy_score(Y[test], y_validation_pred))
    
print("\nAveraging the 5-fold results:")
print("%s: %.2f%%" % ('AVG Train Acc ', numpy.mean(trainScores) * 100))
print("%s: %.2f%%" % ('AVG Validation Acc ', numpy.mean(cvscores) * 100))
print("Validation precision - mean: %f, stddev: %f" % (numpy.mean(val_precisions), numpy.std(val_precisions)))
print("Validation recall - mean: %f, stddev: %f" % (numpy.mean(val_recalls), numpy.std(val_recalls)))
print("Validation f-score - mean: %f, stddev: %f" % (numpy.mean(val_f_scores), numpy.std(val_f_scores)))
print("Confusion matrix:")
print (sum(val_conmats).astype(float) / fold_counter)


Averaging the 5-fold results:
AVG Train Acc : 95.64%
AVG Validation Acc : 90.21%
Validation precision - mean: 0.903377, stddev: 0.051357
Validation recall - mean: 0.901404, stddev: 0.056337
Validation f-score - mean: 0.901083, stddev: 0.042225
Confusion matrix:
[[64.88  6.52]
 [ 6.28 52.92]]
